<a href="https://colab.research.google.com/github/sujitkumar205/RICEVD/blob/main/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text  import CountVectorizer
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from scipy.spatial import distance
import statistics
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import gmean

# ReliefF

In [ ]:
def reliefF(df, number_of_neighbours, instances_to_select, number_of_features):
  features = df.iloc[:,:-1]
  labels = df.iloc[:,-1]
  rows,columns = features.shape

  #initialize weights to zero
  weights = np.zeros(columns,dtype = 'int')

  #unique labels
  unique_labels = np.unique(labels)

  #used to select random instance
  instances=np.array(list(range(1,rows)))

  #difference between maximum and minimum of each feature used to calculate diff
  minimums=np.min(features.values,axis=0)
  maximums=np.max(features.values,axis=0)
  difference=np.subtract(maximums,minimums)

  for i in range(instances_to_select):

    #choose a random instance and remove from instances to avoid selecting same thing again
    random_instance = np.random.choice(instances[:-1])
    instances=np.delete(instances,np.where(instances==random_instance))

    #features of random instance used to calculate diff later on
    random_instance_features = features.iloc[random_instance,:].values

    #label of random instance and probability of label class
    random_instance_label = labels[random_instance]
    probability_random_instance_label = len(np.where(labels==random_instance_label)[0])/rows

    #calculate euclidean distance between random instance and all other instances
    distances = []
    for temp in instances:
      temp_features = features.iloc[temp,:].values
      dist = distance.euclidean(random_instance_features,temp_features)
      distances.append(dist)
    
    #sort instances based on distances
    distances = np.array(distances)
    arr1inds = distances.argsort()
    sorted_distances = distances[arr1inds[::]]
    sorted_instances = instances[arr1inds[::]]

    #initialize list of nearest hits for random instance label and dictionary of nearest misses for every other label
    nearest_hits = []
    nearest_misses = {}

    #finding nearest hits for random instance label
    for temp in sorted_instances:
      if labels[temp] == random_instance_label:
        nearest_hits.append(temp)
      if len(nearest_hits) == number_of_neighbours:
        break
      
    #finding nearest misses for all other labels
    for x in unique_labels:
      if x == random_instance_label:
        continue      
      nearest_misses[x] = []
      for temp in sorted_instances:
        if labels[temp] == x:
          nearest_misses[x].append(temp)
        if len(nearest_misses[x]) == number_of_neighbours:
          break

    #used to find sum of diff function in weights equation for hits
    total_hit = np.zeros(columns,dtype='int')

    #find sum of diff function in weights equation for hits
    for hit in range(len(nearest_hits)):
      hI = features.iloc[nearest_hits[hit],:].values
      dRH = np.divide(np.abs(np.subtract(random_instance_features,hI)),difference)
      dRH = dRH/(instances_to_select * number_of_neighbours)
      total_hit = np.add(total_hit,dRH)

    #used to find sum of diff function in weights equation for misses
    total_miss=np.zeros(columns,dtype='int')

    #find sum of diff function in weights equation for misses in each class
    for each_label in nearest_misses:
      temp_miss=np.zeros(columns,dtype='int')
      pclass=len(np.where(labels==each_label)[0])/rows #getting the probability of getting this class
      postProb=pclass/(1-probability_random_instance_label) #calculating the posterior probability of getting this class

      for each_miss in nearest_misses[each_label]:
        mI = features.iloc[each_miss,:].values
        dRM = np.divide(np.abs(np.subtract(random_instance_features,mI)),difference)
        dRM = dRM/(instances_to_select * number_of_neighbours)
        temp_miss = np.add(temp_miss,dRM)

      total_miss = np.add(total_miss,(temp_miss*postProb))
    
    #update value of weights based on total hits and total miss and diff function values
    weights=np.add(weights,total_miss)
    weights=np.subtract(weights,total_hit) 
    

  #select number_of_features weights with highest values and sort
  ind = np.argpartition(weights, -number_of_features)[-number_of_features:]
  ind = np.sort(ind)[::-1]

  #column names of data frame
  feature_names = list(df.columns.values)
  feature_names = np.array(feature_names)

  #top features based on weights
  top_features = feature_names[ind]

  return top_features



# Chi-Square

In [ ]:
def chi_square(df, number_of_features):
  features = df.iloc[:,:-1]
  labels = df.iloc[:,-1]
  labels=labels.astype('int') 
  test = SelectKBest(score_func=chi2, k=number_of_features)
  fit = test.fit(features, labels)
  chisquare_features = fit.get_feature_names_out(input_features=None)
  return chisquare_features

# SVM-RFE

In [ ]:
def svmrfe(df, number_of_features):
  features = df.iloc[:,:-1]
  labels = df.iloc[:,-1]
  #estimator = SVR(kernel="linear",cache_size=7000)
  #estimator = LinearSVR(max_iter=100000,dual = True)
  estimator = LinearSVC(random_state=0, tol=1e-5)
  selector = RFE(estimator, n_features_to_select=number_of_features, step=10)
  selector = selector.fit(features, labels)
  feature_ranks = list(selector.ranking_)
  feature_names = list(df.columns.values)
  rank_dictionary = dict(zip(feature_names, feature_ranks))
  svmrfe_features = [feature for feature, rank in rank_dictionary.items() if rank == 1]
  return svmrfe_features


# ReliefF Variable Distance

https://docs.scipy.org/doc/scipy/reference/spatial.distance.html

In [ ]:
def calcDistance(random_instance_features, temp_features, distance_variable):
  if distance_variable == 0:
    dist = distance.braycurtis(random_instance_features,temp_features)
    return dist
  if distance_variable == 1:
    dist = distance.canberra(random_instance_features,temp_features)
    return dist
  if distance_variable == 2:
    dist = distance.chebyshev(random_instance_features,temp_features)
    return dist
  if distance_variable == 3:
    dist = distance.cityblock(random_instance_features,temp_features)
    return dist
  if distance_variable == 4:
    dist = distance.correlation(random_instance_features,temp_features)
    return dist
  if distance_variable == 5:
    dist = distance.cosine(random_instance_features,temp_features)
    return dist
  if distance_variable == 6:
    dist = distance.euclidean(random_instance_features,temp_features)
    return dist
  if distance_variable == 7:
    dist = distance.jensenshannon(random_instance_features,temp_features)
    return dist
  if distance_variable == 8:
    dist = distance.sqeuclidean(random_instance_features,temp_features)
    return dist
  return 0

In [ ]:
def reliefF_variable(df, number_of_neighbours, instances_to_select, number_of_features, distance_variable):

  features = df.iloc[:,:-1]
  labels = df.iloc[:,-1]
  rows,columns = features.shape

  #initialize weights to zero
  weights = np.zeros(columns,dtype = 'int')

  #unique labels
  unique_labels = np.unique(labels)

  #used to select random instance
  instances=np.array(list(range(1,rows)))

  #difference between maximum and minimum of each feature used to calculate diff
  minimums=np.min(features.values,axis=0)
  maximums=np.max(features.values,axis=0)
  difference=np.subtract(maximums,minimums)
  
  total_instances = instances_to_select * len(unique_labels)
  label_count = {}

  for i in unique_labels:
    label_count[i] = 0

  for i in range(total_instances):

    #choose a random instance and remove from instances to avoid selecting same thing again
    random_instance = np.random.choice(instances[:-1])
    instances=np.delete(instances,np.where(instances==random_instance))

    #features of random instance used to calculate diff later on
    random_instance_features = features.iloc[random_instance,:].values

    #label of random instance and probability of label class
    random_instance_label = labels[random_instance]
    probability_random_instance_label = len(np.where(labels==random_instance_label)[0])/rows

    if label_count[random_instance_label] >= instances_to_select:
      i = i-1
      continue
    
    else:
      label_count[random_instance_label] = label_count[random_instance_label] + 1

    #calculate euclidean distance between random instance and all other instances
    distances = []
    for temp in instances:
      temp_features = features.iloc[temp,:].values
      dist = calcDistance(random_instance_features, temp_features, distance_variable)
      distances.append(dist)
    
    #sort instances based on distances
    distances = np.array(distances)
    arr1inds = distances.argsort()
    sorted_distances = distances[arr1inds[::]]
    sorted_instances = instances[arr1inds[::]]

    #initialize list of nearest hits for random instance label and dictionary of nearest misses for every other label
    nearest_hits = []
    nearest_misses = {}

    #finding nearest hits for random instance label
    for temp in sorted_instances:
      if labels[temp] == random_instance_label:
        nearest_hits.append(temp)
      if len(nearest_hits) == number_of_neighbours:
        break
      
    #finding nearest misses for all other labels
    for x in unique_labels:
      if x == random_instance_label:
        continue      
      nearest_misses[x] = []
      for temp in sorted_instances:
        if labels[temp] == x:
          nearest_misses[x].append(temp)
        if len(nearest_misses[x]) == number_of_neighbours:
          break

    #used to find sum of diff function in weights equation for hits
    total_hit = np.zeros(columns,dtype='int')

    #find sum of diff function in weights equation for hits
    for hit in range(len(nearest_hits)):
      hI = features.iloc[nearest_hits[hit],:].values
      dRH = np.divide(np.abs(np.subtract(random_instance_features,hI)),difference)
      dRH = dRH/(instances_to_select * number_of_neighbours)
      total_hit = np.add(total_hit,dRH)

    #used to find sum of diff function in weights equation for misses
    total_miss=np.zeros(columns,dtype='int')

    #find sum of diff function in weights equation for misses in each class
    for each_label in nearest_misses:
      temp_miss=np.zeros(columns,dtype='int')
      pclass=len(np.where(labels==each_label)[0])/rows #getting the probability of getting this class
      postProb=pclass/(1-probability_random_instance_label) #calculating the posterior probability of getting this class

      for each_miss in nearest_misses[each_label]:
        mI = features.iloc[each_miss,:].values
        dRM = np.divide(np.abs(np.subtract(random_instance_features,mI)),difference)
        dRM = dRM/(instances_to_select * number_of_neighbours)
        temp_miss = np.add(temp_miss,dRM)

      total_miss = np.add(total_miss,(temp_miss*postProb))
    
    #update value of weights based on total hits and total miss and diff function values
    weights=np.add(weights,total_miss)
    weights=np.subtract(weights,total_hit) 
    

  #select number_of_features weights with highest values and sort
  ind = np.argpartition(weights, -number_of_features)[-number_of_features:]
  ind = np.sort(ind)[::-1]

  #column names of data frame
  feature_names = list(df.columns.values)
  feature_names = np.array(feature_names)

  #top features based on weights
  top_features = feature_names[ind]

  return top_features


In [ ]:
def reliefF_variable_main(df, number_of_neighbours, instances_to_select, number_of_features):

  features = df.iloc[:,:-1]

  braycurtis = []
  canberra = []
  chebyshev = []
  cityblock = []
  correlation = []
  cosine = []
  euclidean = []
  jensenshannon = []
  sqeuclidean = []

  for i in range(len(features)-1):
    for j in range(i+1, len(features)):
      braycurtis.append(distance.braycurtis(features.iloc[i],features.iloc[j]))
      canberra.append(distance.canberra(features.iloc[i],features.iloc[j]))
      chebyshev.append(distance.chebyshev(features.iloc[i],features.iloc[j]))
      cityblock.append(distance.cityblock(features.iloc[i],features.iloc[j]))
      correlation.append(distance.correlation(features.iloc[i],features.iloc[j]))
      cosine.append(distance.cosine(features.iloc[i],features.iloc[j]))
      euclidean.append(distance.euclidean(features.iloc[i],features.iloc[j]))
      jensenshannon.append(distance.jensenshannon(features.iloc[i],features.iloc[j]))
      sqeuclidean.append(distance.sqeuclidean(features.iloc[i],features.iloc[j]))

  standard_deviation = []
  standard_deviation.append(statistics.stdev(braycurtis))
  standard_deviation.append(statistics.stdev(canberra))
  standard_deviation.append(statistics.stdev(chebyshev))
  standard_deviation.append(statistics.stdev(cityblock))
  standard_deviation.append(statistics.stdev(correlation))
  standard_deviation.append(statistics.stdev(cosine))
  standard_deviation.append(statistics.stdev(euclidean))
  standard_deviation.append(statistics.stdev(jensenshannon))
  standard_deviation.append(statistics.stdev(sqeuclidean))

  max_value = max(standard_deviation)
  distance_variable = standard_deviation.index(max_value)

  print(distance_variable)
  #distance_variable = 1
  features_combined = []

  for i in range(10):
    a = reliefF_variable(df, number_of_neighbours, instances_to_select, number_of_features, distance_variable)
    features_combined = features_combined + list(a)

  features_count = Counter(features_combined)

  features_count_sorted = sorted(features_count.items(), key=lambda x: x[1], reverse=True)

  variable_distance_relieff_features = []
  for i in range(number_of_features):
    variable_distance_relieff_features.append(features_count_sorted[i][0])

  return variable_distance_relieff_features

# Comparing Feature Selection Algorithms

## Control Function

In [ ]:
#@title
def normalize_data(dataframe):
  dataframe=(dataframe-dataframe.mean())/dataframe.std()
  dataframe=(dataframe-dataframe.min())/(dataframe.max()-dataframe.min())
  dataframe[np.isnan(dataframe)] = 0
  return dataframe

In [ ]:
#@title
def plot_roc_auc(x_test, y_test, model):
  #define metrics
  y_pred_proba = model.predict_proba(x_test)[::,1]
  y_pred_proba = normalize_data(y_pred_proba)
  fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
  auc = metrics.roc_auc_score(y_test, y_pred_proba)
  rocauc_score = metrics.roc_auc_score(y_test, y_pred_proba)
  #print('The ROCAUC is {}'.format(rocauc_score))


  #create ROC curve
  #plt.plot(fpr,tpr,label="AUC="+str(auc))
  #plt.ylabel('True Positive Rate')
  #plt.xlabel('False Positive Rate')
  #plt.legend(loc=4)
  #plt.show()

  return rocauc_score

In [ ]:
#@title
def training_model(train, test, fold_no, model):
  x_train = train.drop(['label'],axis=1)
  y_train = train.label
  x_test = test.drop(['label'],axis=1)
  y_test = test.label
  model.fit(x_train, y_train)
  score = model.score(x_test,y_test)
  scores_list = []

  #print('For Fold {} the accuracy is {}'.format(str(fold_no),score))
  scores_list.append(score)

  f1_scores = f1_score(y_test,model.predict(x_test))
  #print('F1 score is {}'.format(f1_scores))
  scores_list.append(f1_scores)

  unique_labels = np.unique(y_test)
  predicted = model.predict(x_test)
  #y_test = list(y_test)
  #print(y_test)

  '''

  recall = []
  for k in unique_labels:
    indices = [int(i) for i, x in enumerate(y_test) if x == k]
    res_list_pred = list(map(predicted.__getitem__, indices))
    res_list_test = list(map(list(y_test).__getitem__, indices))
    recall_class = recall_score(res_list_pred, res_list_test)
    recall.append(recall_class)
  
  recall = [0.000001 if x == 0 else x for x in recall]
  geometric_mean = gmean(recall)

  '''


  geometric_mean = geometric_mean_score(y_test,model.predict(x_test))
  #print('The geometric mean is {}'.format(geometric_mean))
  scores_list.append(geometric_mean)

  roc_scores = plot_roc_auc(x_test, y_test,model)
  scores_list.append(roc_scores)

  return scores_list

In [ ]:
def control_function(dataframe, n_of_splits, df_name, path_name, df_scores):
  #dataframe = dataframe.reset_index()
  #dataframe = dataframe.replace(np.inf, dataframe.mean())
  #dataframe = dataframe.fillna(dataframe.mean())
  skf = StratifiedKFold(n_splits=n_of_splits)
  x = dataframe
  y = dataframe.label

  svc_model = SVC(kernel='linear',probability=True)
  dtc_model = DecisionTreeClassifier(random_state = 0)
  rfc_model = RandomForestClassifier(random_state=0)
  naive_bayes_model = GaussianNB()
  gradient_boosting_model = GradientBoostingClassifier(random_state=0)
  knn_model = KNeighborsClassifier(n_neighbors=5)

  models = [svc_model, dtc_model, rfc_model, naive_bayes_model, gradient_boosting_model, knn_model]
  models_name = ['Support Vector Classifier', 'Decision Tree Classifier', 'Random Forest Classifier', 'Gaussian Naive Bayes', 'Gradient Boosting Classifier', 'K Nearest Neighbour']


  for i in range(len(models)):
    print(models_name[i], " + ", df_name, " + ", path_name, "\n")    
    fold_no = 1
    for train_index,test_index in skf.split(x, y):
      train = dataframe.iloc[train_index,:]
      test = dataframe.iloc[test_index,:]
      model_scores = training_model(train, test, fold_no, models[i])
      df_scores.loc[0 if pd.isnull(df_scores.index.max()) else df_scores.index.max() + 1] = [df_name, path_name, models_name[i], fold_no] + model_scores
      fold_no += 1
      #print("\n")

In [ ]:
def normalize_dataframe(dataframe):
  dataframe=(dataframe-dataframe.mean())/dataframe.std()
  dataframe=(dataframe-dataframe.min())/(dataframe.max()-dataframe.min())
  dataframe.fillna(0,inplace=True)
  return dataframe

In [ ]:
def main_function(path_name, df_scores, feature_names_path, count):
  df = pd.read_csv(path_name)

  print(df.shape)

  number_of_neighbours = 5
  instances_to_select = 10
  number_of_features = 100

  
  df = normalize_dataframe(df)

  #Chi-Square
  chisquare_features = chi_square(df, number_of_features)
  df_chisquare = pd.read_csv(path_name, usecols = chisquare_features)
  df_chisquare = normalize_dataframe(df_chisquare)
  

  #ReliefF
  reliefF_features = reliefF(df, number_of_neighbours, instances_to_select, number_of_features)
  df_reliefF = pd.read_csv(path_name, usecols = reliefF_features)
  df_reliefF = normalize_dataframe(df_reliefF)
  

  #SVM-RFE
  svmrfe_features = svmrfe(df, number_of_features)
  df_svmrfe = pd.read_csv(path_name, usecols = svmrfe_features)
  df_svmrfe = normalize_dataframe(df_svmrfe)
  

  #Variable-ReliefF
  variable_distance_relieff_features = reliefF_variable_main(df, number_of_neighbours, instances_to_select, number_of_features)
  df_variable_reliefF = pd.read_csv(path_name, usecols = variable_distance_relieff_features)
  df_variable_reliefF = normalize_dataframe(df_variable_reliefF)
  


  #Original
  df_original = pd.read_csv(path_name).iloc[:,:-1]
  df_original = normalize_dataframe(df_original)

  features = pd.DataFrame(
    {'svmrfe': svmrfe_features,
     'chisquare': chisquare_features,
     'relief': reliefF_features,
     'variable': variable_distance_relieff_features
    })
  feature_names_path = feature_names_path + str(count)
  features.to_csv(feature_names_path)


  #Labels
  df_labels = pd.read_csv(path_name).iloc[:,-1]
  df_original['label'] = df_labels
  df_reliefF['label'] = df_labels
  df_chisquare['label'] = df_labels
  df_svmrfe['label'] = df_labels
  df_variable_reliefF['label'] = df_labels

  n_of_splits = 5

  df_list = [df_original, df_reliefF, df_chisquare, df_svmrfe, df_variable_reliefF]
  df_list_name = ['Original', 'ReliefF', 'ChiSquare', 'SVMRFE', 'Variable ReliefF']

  for df_index in range(len(df_list)):
    control_function(df_list[df_index], n_of_splits, df_list_name[df_index], path_name, df_scores)

In [ ]:
path_name_list = ['/content/gastroenterology.csv','/content/leukemia.csv', '/content/colon 2000.csv', '/content/DLBCL.csv', '/content/LSVT_voice_rehabilitation.csv', '/content/gastric cancer.csv',]
#path_name_list = ['/content/staDynBenignLab.csv','/content/qsar_androgen_receptor.csv','/content/qsar_oral_toxicity.csv']
df_scores = pd.DataFrame(columns=['Algorithm','Dataset','Model','Fold Number', 'Accuracy','F1','Geometric Mean','AUC'])
feature_names_path = 'featuresLow.csv'
count = 1
for path_name in path_name_list:
  main_function(path_name, df_scores,feature_names_path, count)
  count = count + 1

(152, 699)
Support Vector Classifier  +  Original  +  /content/gastroenterology.csv 

Decision Tree Classifier  +  Original  +  /content/gastroenterology.csv 

Random Forest Classifier  +  Original  +  /content/gastroenterology.csv 

Gaussian Naive Bayes  +  Original  +  /content/gastroenterology.csv 

Gradient Boosting Classifier  +  Original  +  /content/gastroenterology.csv 

K Nearest Neighbour  +  Original  +  /content/gastroenterology.csv 

Support Vector Classifier  +  ReliefF  +  /content/gastroenterology.csv 

Decision Tree Classifier  +  ReliefF  +  /content/gastroenterology.csv 

Random Forest Classifier  +  ReliefF  +  /content/gastroenterology.csv 

Gaussian Naive Bayes  +  ReliefF  +  /content/gastroenterology.csv 

Gradient Boosting Classifier  +  ReliefF  +  /content/gastroenterology.csv 

K Nearest Neighbour  +  ReliefF  +  /content/gastroenterology.csv 

Support Vector Classifier  +  ChiSquare  +  /content/gastroenterology.csv 

Decision Tree Classifier  +  ChiSquare 

In [ ]:
df_scores.to_csv('low_instance_binary.csv')

In [ ]:
#path_name_list = ['/content/gastroenterology.csv','/content/leukemia.csv', '/content/colon 2000.csv', '/content/DLBCL.csv', '/content/LSVT_voice_rehabilitation.csv', '/content/gastric cancer.csv',]
path_name_list = ['/content/staDynBenignLab.csv','/content/qsar_androgen_receptor.csv','/content/qsar_oral_toxicity.csv']
df_scores = pd.DataFrame(columns=['Algorithm','Dataset','Model','Fold Number', 'Accuracy','F1','Geometric Mean','AUC'])
feature_names_path = 'featuresHigh.csv'
count = 1
for path_name in path_name_list:
  main_function(path_name, df_scores, feature_names_path)
  count = count + 1

In [ ]:
df_scores.to_csv('high_instance_binary.csv')